In [ ]:
import pandas as pd
import numpy as np

# Read the original CSV file
df = pd.read_csv('../../data/accions.csv')

In [ ]:
# Group by 'Sessio' and aggregate the 'Accio' column into a list
grouped_df = df.groupby('Sessio').agg({'Accio': list, 'Tramit': list, 'Usuari': list, 'Data': list}).reset_index()


# Write the grouped data to a new CSV file
grouped_df.to_csv('../../data/grouped_accions.csv', index=False)

In [ ]:
grouped_df['Usuari'] = grouped_df['Usuari'].apply(lambda x: x[-1] if isinstance(x, list) and x else x)

In [ ]:
# Write the grouped data to a new CSV file
grouped_df.to_csv('../../data/grouped_accions_usuari.csv', index=False)

In [ ]:
# grouped_df = pd.read_csv('../../data/grouped_accions_usuari.csv')
(grouped_df['Tramit'].apply(lambda x: len((x))) == 1).sum()

In [ ]:
filtered_df = grouped_df[~((grouped_df['Usuari'].isna()) & (grouped_df['Tramit'].apply(lambda x: len((x))) == 1))]

In [ ]:
# Write the grouped data to a new CSV file
filtered_df.to_csv('../../data/grouped_accions_filtered.csv', index=False)

In [ ]:
filtered_df = pd.read_csv('../../data/grouped_accions_filtered.csv')

In [ ]:
final_df = filtered_df[~(filtered_df['Tramit'].apply(lambda x: len(eval(x))) == 1)]

In [ ]:
final_df = final_df.drop(columns=['Usuari', 'Accio', 'Data', 'Sessio'])

In [ ]:
final_df.to_csv('../../data/grouped_accions_final.csv', index=False)

In [ ]:
final_df = pd.read_csv('../../data/grouped_accions_final.csv')

#### 


In [ ]:
final_df['Last_Tramit'] = final_df['Tramit'].apply(lambda x: eval(x)[-1] if isinstance(eval(x), list) and eval(x) else x)

In [ ]:
final_df['Tramit'] = final_df['Tramit'].apply(lambda x: str(eval(x)[:-1]) if isinstance(eval(x), list) and eval(x) else x)

In [ ]:
final_df.to_csv('../../data/final_dataset.csv', index=False)

In [ ]:
train_final_df = pd.read_csv('../../data/final_dataset.csv')

In [ ]:
X_train = train_final_df['Tramit']
y_train = train_final_df['Last_Tramit']



In [ ]:
# Crear un dataset de muestra con 50 filas
sample_df = train_final_df.sample(n=50, random_state=42)

# Guardar el dataset de muestra en el directorio
sample_df.to_csv('../../data/sample_train_final_df.csv', index=False)


In [ ]:
# Import the sample_tramits.csv file
sample_tramits_df = pd.read_csv('../../data/sample_tramits.csv')

# Erase the 'Vigent' column
sample_tramits_df = sample_tramits_df.drop(columns=['Vigent'])

# Display the first few rows of the dataframe to verify
print(sample_tramits_df.head())

sample_tramits_df.to_csv('../../data/sample_tramits_df.csv', index=False)

In [ ]:

tramits_df = pd.read_csv('../../data/tramits.csv')


tramits_df['Sequence'] = range(502)


tramits_df.to_csv('../../data/tramits.csv', index=False)

In [ ]:
id_to_sequence = dict(zip(tramits_df['Id'], tramits_df['Sequence']))

final_df['Tramit'] = final_df['Tramit'].apply(lambda x: [id_to_sequence.get(tramit, tramit) for tramit in eval(x)])
final_df['Last_Tramit'] = final_df['Last_Tramit'].apply(lambda x: id_to_sequence.get(x, x))

final_df.to_csv('../../data/final_dataset_mapped.csv', index=False)



In [ ]:

final_df = pd.read_csv('../../data/final_dataset_mapped.csv')
final_df = final_df[final_df['Tramit'].apply(len) <= 30]



In [ ]:
print("Distribución de y_train:", pd.Series(y_train).value_counts())

print("Distribución del 82 en y_train:", (y_train == 314).sum())


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, LayerNormalization, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split  # Para dividir datos en train/test
import numpy as np
import pandas as pd
import ast  # Para usar literal_eval y evitar eval() peligroso

# ------------------ Parámetros del Modelo ------------------
max_seq_len = 20  # Longitud máxima de la secuencia de trámites
vocab_size = 502  # Cantidad total de trámites posibles (codificados)
embed_dim = 32  # Dimensión del embedding
num_heads = 2  # Cantidad de cabezas de atención
ff_dim = 64  # Dimensión de la red Feed-Forward en el Transformer
num_transformer_blocks = 1  # Número de bloques de Transformer
dropout_rate = 0.1  # Tasa de dropout

# ------------------ Carga y Procesamiento del archivo CSV ------------------
# Leer los datos de training.csv
file_path = "../../data/final_dataset_mapped.csv"  # Cambia esta ruta si el CSV está en otra ubicación
data = pd.read_csv(file_path)

# Procesar columna 'Tramit'
# Convertimos las cadenas tipo lista a listas reales de Python
data['Tramit'] = data['Tramit'].apply(lambda x: ast.literal_eval(x))

# Validar que los datos no están vacíos
if data.isnull().values.any():
    raise ValueError("El archivo CSV contiene valores nulos, revisa tu archivo.")

# Asegurarnos de que las etiquetas están en formato entero
data['Last_Tramit'] = data['Last_Tramit'].astype(int)

# Extraer las columnas procesadas
X_raw = data['Tramit'].tolist()  # Lista de listas de trámites para X_data
y_data = data['Last_Tramit'].values  # Etiquetas para y_data

# Padding para X_data (ajustamos las listas a max_seq_len)
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_data = pad_sequences(X_raw, maxlen=max_seq_len, padding='post', truncating='post')

# Dividir los datos en entrenamiento (80%) y prueba (20%)
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

# ------------------ Construcción del Modelo con Transformer ------------------
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential(
            [Dense(ff_dim, activation="relu"), Dense(embed_dim)]
        )
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions


# Crear el modelo completo
def create_model(vocab_size, max_seq_len, embed_dim, num_heads, ff_dim, num_transformer_blocks, dropout_rate):
    inputs = Input(shape=(max_seq_len,))
    embedding_layer = TokenAndPositionEmbedding(max_seq_len, vocab_size, embed_dim)
    x = embedding_layer(inputs)
    for _ in range(num_transformer_blocks):
        x = TransformerBlock(embed_dim, num_heads, ff_dim, rate=dropout_rate)(x)
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = Dropout(dropout_rate)(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(vocab_size, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=outputs)
    return model


# Callbacks para detener entrenamiento innecesario
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_accuracy', patience=250, restore_best_weights=True)
class PrintEpochCallback(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        print(f"Epoch {epoch + 1} starting...")

# Instanciar el modelo
model = create_model(vocab_size, max_seq_len, embed_dim, num_heads, ff_dim, num_transformer_blocks, dropout_rate)

# Compilar el modelo
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Resumen del modelo
model.summary()

# Entrenar el modelo
history = model.fit(
    X_train, y_train,
    batch_size=32,
    epochs=3,  # Epochs ajustados para inicializar el entrenamiento
    validation_data=(X_test, y_test),  # Validación con los datos de test explícitos
    callbacks=[PrintEpochCallback(), early_stop]  # EarlyStopping configurado
)

# ------------------ Evaluación en el Test Set ------------------
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Loss en Test: {test_loss}")
print(f"Accuracy en Test: {test_accuracy}")



In [ ]:
print("Ejemplo de X_train:", X_train[:5])
print("Ejemplo de y_train:", y_train[:5])
print("Máximo valor en X_train:", np.max(X_train))
print("Máximo valor en y_train:", np.max(y_train))


In [ ]:
print("Valores únicos en y_train:", np.unique(y_train))


In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.legend()
plt.show()


In [ ]:
# --------------------- Realizar predicciones con input RAW ----------------------

# Ejemplo de nueva secuencia "RAW" (sin padding)
raw_input_sequence = [364]  # Secuencia directa del usuario

# Realizamos el padding para ajustar al tamaño `max_seq_len` del modelo
from tensorflow.keras.preprocessing.sequence import pad_sequences
input_sequence = pad_sequences([raw_input_sequence], maxlen=max_seq_len, padding='post', truncating='post')

# Realizar la predicción
predictions = model.predict(input_sequence)

# Obtener el índice y probabilidad del trámite con la mayor puntuación
recommended_tramit_index = np.argmax(predictions)
recommended_tramit_probability = predictions[0][recommended_tramit_index]

# Obtener las 10 mejores predicciones (índices y probabilidades) ordenadas
# np.argsort devuelve índices ordenados, usamos [-10:] para las mejores 10
top_10_indices = np.argsort(predictions[0])[-10:][::-1]  # Índices de las 10 mejores categorías (orden descendente)
top_10_probabilities = predictions[0][top_10_indices]  # Probabilidades correspondientes

# Crear una lista con los pares (índice, probabilidad)
top_10_predictions = list(zip(top_10_indices, top_10_probabilities))

# Mostrar los resultados
print("\nTrámite recomendado:", recommended_tramit_index)
print("Probabilidad del trámite recomendado:", recommended_tramit_probability)
print("\nTop 10 mejores predicciones (categoría, probabilidad):")
for category, prob in top_10_predictions:
    print(f"Trámite: {category}, Probabilidad: {prob}")


In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# --------------------- Cargar datos de 'tramits.csv' ----------------------
# Asegúrate de que la columna `Vigent` y los índices estén correctamente mapeados
tramits_df = pd.read_csv('../../data/tramits.csv')  # Carga el archivo tramits.csv
vigent_mask = tramits_df['Vigent'] == True  # Filtrar los trámites con Vigent=True
vigent_indices = tramits_df[vigent_mask].index.tolist()  # Índices de los trámites válidos

# --------------------- Realizar predicciones con filtro ----------------------
# Ejemplo de nueva secuencia "RAW" (sin padding)
raw_input_sequence = [364]  # Entrada del usuario

# Padding para ajustar al tamaño `max_seq_len` del modelo
input_sequence = pad_sequences([raw_input_sequence], maxlen=max_seq_len, padding='post', truncating='post')

# Realizar la predicción
predictions = model.predict(input_sequence)

# Crear un array con probabilidades solo para los índices donde Vigent=True
filtered_predictions = {idx: predictions[0][idx] for idx in vigent_indices}

# Ordenar los índices por probabilidad (descendente)
sorted_filtered_predictions = sorted(filtered_predictions.items(), key=lambda x: x[1], reverse=True)

# Obtener el trámite con la mayor probabilidad
recommended_tramit_index, recommended_tramit_probability = sorted_filtered_predictions[0]

# Crear una lista con las 10 mejores predicciones filtradas
top_10_filtered_predictions = sorted_filtered_predictions[:10]

# --------------------- Mostrar resultados ----------------------
print("Predicciones filtradas por 'Vigent=True':")
for category, prob in top_10_filtered_predictions:
    print(f"Trámite: {category}, Probabilidad: {prob}")

print("\nTrámite recomendado:", recommended_tramit_index)
print("Probabilidad del trámite recomendado:", recommended_tramit_probability)


In [ ]:
model_name = "modelo_transformer_tramites.h5"
model.save(model_name)
print(f"Modelo guardado como {model_name}")

In [ ]:
from tensorflow.keras.models import load_model

# Cargar el modelo guardado
loaded_model = load_model("modelo_transformer_tramites.h5")
print("Modelo cargado correctamente.")

# Usar el modelo cargado para predicciones
new_sequence = np.random.randint(1, vocab_size, (1, max_seq_len))
print(new_sequence)
predictions = loaded_model.predict(new_sequence)

print("Predicciones del modelo cargado:", predictions)
